# Evaluate Prompt Flows for "Simplify Language"

This notebooks uses the Promtflow SDK to run different variants and configurations of the "simplify" flow, and evaluate the results. You will be able to assess the relative quality and cost of different LLM models, prompt templates, parameter configurations etc. This enables an interactive, iterative cycle to evaluate and improve your flows.

In [11]:
%%capture
%pip install -r ../eval-simplify/requirements.txt
!python -m spacy download de_core_news_sm

In [2]:
%reload_ext autoreload
%autoreload 2

import json, os
from promptflow.client import PFClient
from promptflow.entities import AzureOpenAIConnection, OpenAIConnection
from IPython.display import JSON, Markdown
import logging
import pandas as pd


# Helper function to prepare configs for different flow configurations
from helpers import create_flow_config, execute_run, result_markdown,compare_simplified, execute_eval_run


#Supress default INFO logging
os.environ["PF_LOGGING_LEVEL"] = "ERROR"

# The required connection name (see flow.dag.yaml)
# Currently, must be OpenAI or Azure OpenAI connection.
conn_name = "openai_connection"

# Paths to the the flow and eval flow directory
flow_path="../flow-simplify-es"
eval_flow_path="../eval-simplify"

# Path to the test data
data_path="../test_data.jsonl"





In [3]:
# Create the Promptflow client
pf = PFClient()

# Initialize the connection
conn = None


# Check if the connection already exists (e.g. already created, either here or via CLI)
for c in pf.connections.list():
    if (c.name == conn_name) and (c.type == "OpenAI"):
        conn = pf.connections.get(name=conn_name)


if (conn is None):
    print(f"Connection '{conn_name}' not found, trying to create...")
    try:
        # use this for Azure OpenAI
        # Follow https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal to create an Azure Open AI resource.
        #connection = AzureOpenAIConnection(
        #    name=conn_name,
        #    api_key="<user-input>",
        #    api_base="<test_base>",
        #    api_type="azure",
        #    api_version="<test_version>",
        #)
        
        # use this if you have an existing OpenAI account
        connection = OpenAIConnection(
            name=conn_name,
            api_key=os.environ.get("OPENAI_API_KEY") or "[ADD YOUR OPENAI API KEY HERE]",
        )
        conn = pf.connections.create_or_update(connection)
    except Exception as e:
        print(f"Failed to create connection: {e}")

if (conn):
    print (f"Connection '{conn_name}' is ready for use")
else: 
    print (f"Could ot find or create required connection, try setting the OPENAI_API_KEY environment variable.")

Connection 'openai_connection' is ready for use


### Test the flow

In [4]:
output = pf.flows.test(
    flow_path,
    inputs={
        "original_text": "Dies ist ein einfacher Text, um die Funktionalität zu testen. Er ist nicht besonders lang, und nicht in 'Behördendeutsch', aber sollte ausreichen.",
    },
    stream=False,
)

JSON(output, expanded=True)

Starting prompt flow service...
Start prompt flow service on 127.0.0.1:23333, version: 1.13.0.
You can stop the prompt flow service with the following command:'pf service stop'.

2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Start to run 4 nodes with concurrency level 16.
2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Executing node rules. node run id: ea7037d0-23fc-4e27-9012-da1c8fc1dc89_rules_0
2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Executing node completeness. node run id: ea7037d0-23fc-4e27-9012-da1c8fc1dc89_completeness_0
2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Node rules completes.
2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Node completeness completes.
2024-07-07 21:41:46 +0000     871 execution.flow     INFO     Executing node create_prompt. node run id: ea7037d0-23fc-4e27-9012-da1c

<IPython.core.display.JSON object>

### Run different variants of the '__simplify_es__' flow
The following cells create some different variants of the "simplify_es" flow, following a simple naming convention. 
Existing runs with the same name will be overwritten. See output for links to trace information.

We use the `prompflow.client` SDK to create runs, see `helpers.py` for details.

#### Step 1: Create the flow configurations

In [5]:
gpt4o_config = create_flow_config(model="gpt-4o")
gpt35turbo_config = create_flow_config(model="gpt-3.5-turbo")
gpt35turbo_config_2 = create_flow_config(model="gpt-3.5-turbo", variant="${rules.short_instructions}")



#### Step 2: Run the flows 
Run the flows using the configs from above (this will take a few moments). 
See output below to open trace information.

In [6]:
result_gpt4o = execute_run (pf, gpt4o_config)
result_gpt35turbo = execute_run (pf, gpt35turbo_config)
result_gpt35turbo_2 = execute_run (pf, gpt35turbo_config_2)


[2024-07-07 21:41:50 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run run_simplify_es_gpt-4o_default, log path: /root/.promptflow/.runs/run_simplify_es_gpt-4o_default/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=run_simplify_es_gpt-4o_default
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0xc2694c557854302288aea6fd9622f274
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x6b2bb6cf4160b0af9aabfb6f88244459


[2024-07-07 21:41:57 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run run_simplify_es_gpt-3.5-turbo_default, log path: /root/.promptflow/.runs/run_simplify_es_gpt-3.5-turbo_default/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=run_simplify_es_gpt-3.5-turbo_default
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x1878ecc3281a99acad6cabba6fa4136e
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0xad9aca8607987224ccc854122b12180c


[2024-07-07 21:42:05 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run run_simplify_es_gpt-3.5-turbo_rules.short_instructions, log path: /root/.promptflow/.runs/run_simplify_es_gpt-3.5-turbo_rules.short_instructions/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=run_simplify_es_gpt-3.5-turbo_rules.short_instructions
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x6d301356c4ff686ff0decd29e1718168
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0xd96a6a9aef076eb0cf6ee1dd8925265b


In [7]:
display(result_markdown(result_gpt4o))
display(result_markdown(result_gpt35turbo))
display(result_markdown(result_gpt35turbo_2))

##### Results for __"run_simplify_es_gpt-4o_default"__
- __Model__: gpt-4o
- __Variant__: None
- __Duration__: 7.172132 seconds
- __Prompt tokens__: 2573
- __Completion tokens__: 336
- __Metrics__: {}

##### Results for __"run_simplify_es_gpt-3.5-turbo_default"__
- __Model__: gpt-3.5-turbo
- __Variant__: None
- __Duration__: 7.113703 seconds
- __Prompt tokens__: 3012
- __Completion tokens__: 269
- __Metrics__: {}

##### Results for __"run_simplify_es_gpt-3.5-turbo_rules.short_instructions"__
- __Model__: gpt-3.5-turbo
- __Variant__: ${rules.short_instructions}
- __Duration__: 6.107623 seconds
- __Prompt tokens__: 804
- __Completion tokens__: 230
- __Metrics__: {}

#### Step 4: Compare simplified texts

A first, visual comparison of the different simplified versions of the text.

In [8]:
compare_simplified([result_gpt4o,result_gpt35turbo,result_gpt35turbo_2])


,inputs.original_text,run_simplify_es_gpt-4o_default,run_simplify_es_gpt-3.5-turbo_default,run_simplify_es_gpt-3.5-turbo_rules.short_instructions
0,"Als Vernehmlassungsverfahren wird diejenige Phase innerhalb des Vorverfahrens der Gesetzgebung bezeichnet, in der Vorhaben des Bundes von erheblicher politischer, finanzieller, wirtschaftlicher, ökologischer, sozialer oder kultureller Tragweite auf ihre sachliche Richtigkeit, Vollzugstauglichkeit und Akzeptanz hin geprüft werden. Die Vorlage wird zu diesem Zweck den Kantonen, den in der Bundesversammlung vertretenen Parteien, den Dachverbänden der Gemeinden, Städte und der Berggebiete, den Dachverbänden der Wirtschaft sowie weiteren, im Einzelfall interessierten Kreisen unterbreitet.","Das Vernehmlassungsverfahren ist eine wichtige Phase bei der Gesetzgebung. In dieser Phase prüft der Bund seine Pläne. Diese Pläne betreffen Politik, Finanzen, Wirtschaft, Umwelt, Soziales oder Kultur. Der Bund will wissen, ob die Pläne richtig sind. Er fragt auch, ob die Pläne durchführbar sind und ob die Menschen sie gut finden. \n\nDazu teilt der Bund die Pläne den Kantonen mit. Kantone sind die grossen Regionen der Schweiz. Der Bund teilt die Pläne auch den Parteien im Parlament mit. Auch die Dachverbände der Gemeinden, Städte und Berggebiete erhalten die Pläne. Die Dachverbände der Wirtschaft bekommen die Pläne ebenfalls. Manchmal interessieren sich noch andere Gruppen für die Pläne. Auch diese anderen Gruppen bekommen dann die Pläne.","Im Vernehmlassungsverfahren wird geprüft, ob Bundesvorhaben inhaltlich und praktisch umsetzbar sind. Die Vorlage wird Kantonen, Parteien, Gemeinde- und Wirtschaftsverbänden sowie anderen Interessengruppen zur Stellungnahme vorgelegt.","In der Vernehmlassung werden wichtige Bundesvorhaben von verschiedenen Gruppen überprüft. Dazu gehören Kantone, Parteien, Gemeindeverbände und Wirtschaftsverbände. Man prüft, ob die Vorhaben politisch, finanziell, wirtschaftlich, ökologisch, sozial oder kulturell sinnvoll sind."
1,"Das in Artikel 26 aPartG vorgesehene Ehehindernis der eingetragenen Partnerschaft wurde aufgehoben und in Artikel 96 nZGB eingearbeitet. Es ist zu präzisieren, dass das Ehehindernis nur dann besteht, wenn eine frühere Ehe oder eine mit einer Drittperson begründete eingetragene Partnerschaft noch besteht bzw. nicht für ungültig erklärt oder aufgelöst worden ist. Folglich können zwei Personen, die nach Inkrafttreten der ZGB-Revision, d.h. nach dem 30. Juni 2022, im Ausland miteinander eine eingetragene Partnerschaft begründet haben, einander heiraten, sofern die allgemeinen Voraussetzungen von Artikel 43 f. IPRG und des materiellen Schweizer Eherechts erfüllt sind, ohne vorgängig ihre eingetragene Partnerschaft auflösen zu müssen (Bericht RK-N, Ziff. 5.3.3 und 7.1, Ausführungen zu Art. 96 E-ZGB).","Das Gesetz wurde geändert. Früher gab es ein Eheverbot für Personen in einer eingetragenen Partnerschaft. Dieses Verbot steht jetzt in Artikel 96 im Zivilgesetzbuch.\n\nDas Gesetz sagt jetzt: Eine Person darf nur heiraten, wenn keine frühere Ehe oder Partnerschaft besteht. Eine alte Ehe oder Partnerschaft muss entweder aufgehoben oder für ungültig erklärt sein.\n\nSeit dem 30. Juni 2022 ist etwas Neues dazugekommen. Personen, die im Ausland eine eingetragene Partnerschaft geschlossen haben, dürfen trotzdem heiraten. Dafür müssen sie die Regeln in den Artikeln 43 und 96 im Zivilgesetzbuch beachten.\n\nDie Partner müssen die Schweizer Ehe-Regeln beachten. Die Partnerschaft muss nicht vorher aufgelöst werden.\n\nDiese Änderungen stehen in einem Bericht des Nationalrats.","In der Schweiz wurde das Ehehindernis für eingetragene Partnerschaften im Gesetz geändert. Früher war dies in Artikel 26 des Partnerschaftsgesetzes geregelt, nun ist es in Artikel 96 des Zivilgesetzbuches zu finden. Das bedeutet: Man darf heiraten, auch wenn eine frühere Partnerschaft noch besteht oder nicht offiziell aufgelöst wurde. Ab dem 30. Juni 2022 können zwei Personen, die im Ausland eine

## Evaluate and Results in Detail

#### Step 1: Run evaluation flows against the runs
This uses the ouputs from the previous runs, as well as original data.  

In [9]:

eval_gpt4o = execute_eval_run(pf, gpt4o_config, result_gpt4o)
eval_gpt35turbo = execute_eval_run(pf, gpt35turbo_config, result_gpt35turbo)
eval_gpt35turbo_2 = execute_eval_run(pf, gpt35turbo_config_2, result_gpt35turbo_2)



[2024-07-07 21:42:12 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run eval_run_simplify_es_gpt-4o_default, log path: /root/.promptflow/.runs/eval_run_simplify_es_gpt-4o_default/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=eval_run_simplify_es_gpt-4o_default
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x53f7330ab73e16553380094f53f3fab0
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x86a193f1c0a018576a5baf6af49cad27
2024-07-07 21:42:17 +0000     871 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-07-07 21:42:17 +0000     871 execution.bulk     INFO     Set process count to 2 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 2}.
2024-07-07 21:42:22 +0000     871 execution.bulk     INFO     Process name(ForkProcess-8:1)-Process id(1561)-Line number(0) start execution.
2024-07-07 21:42:22 +0000     871 execution.bul

[2024-07-07 21:42:28 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run eval_run_simplify_es_gpt-3.5-turbo_default, log path: /root/.promptflow/.runs/eval_run_simplify_es_gpt-3.5-turbo_default/logs.txt


You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0xe6f3659e228516773d8185d83ad5cdbc
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x42121d8d374c254487f2b6b428b2d0a8
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0xfec57b501945422d4779917596f8afd2


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=eval_run_simplify_es_gpt-3.5-turbo_default
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x644f9107bb032af6e0a21522d081be2c
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x936c8098a8a2f3b6beb29210511ae91d
2024-07-07 21:42:32 +0000     871 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-07-07 21:42:32 +0000     871 execution.bulk     INFO     Set process count to 2 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 2}.
2024-07-07 21:42:37 +0000     871 execution.bulk     INFO     Process name(ForkProcess-10:2)-Process id(1745)-Line number(1) start execution.
2024-07-07 21:42:37 +0000     871 execu

[2024-07-07 21:42:41 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run eval_run_simplify_es_gpt-3.5-turbo_rules.short_instructions, log path: /root/.promptflow/.runs/eval_run_simplify_es_gpt-3.5-turbo_rules.short_instructions/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=eval_run_simplify_es_gpt-3.5-turbo_rules.short_instructions
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0xfc5e1d24b03d4dc85a294c253352e6ae
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=flow-simplify-es&uiTraceId=0x8e7b73f688842cd62634a6a64b62a452
2024-07-07 21:42:44 +0000     871 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-07-07 21:42:44 +0000     871 execution.bulk     INFO     Set process count to 2 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 2}.
2024-07-07 21:42:50 +0000     871 execution.bulk     INFO     Process name(ForkProcess-12:1)-Process id(1899)-Line number(1) start execution.
2024-07-07 21:42:50 +0

In [10]:
display(eval_gpt4o["details"][["inputs.model", "inputs.original_text", "inputs.simplified_text", "outputs.zix_score_original", "outputs.zix_score_simplified", "outputs.level_original", "outputs.level_simplified"]])

,inputs.model,inputs.original_text,inputs.simplified_text,outputs.zix_score_original,outputs.zix_score_simplified,outputs.level_original,outputs.level_simplified
0,gpt-4o,"Als Vernehmlassungsverfahren wird diejenige Phase innerhalb des Vorverfahrens der Gesetzgebung bezeichnet, in der Vorhaben des Bundes von erheblicher politischer, finanzieller, wirtschaftlicher, ökologischer, sozialer oder kultureller Tragweite auf ihre sachliche Richtigkeit, Vollzugstauglichkeit und Akzeptanz hin geprüft werden. Die Vorlage wird zu diesem Zweck den Kantonen, den in der Bundesversammlung vertretenen Parteien, den Dachverbänden der Gemeinden, Städte und der Berggebiete, den Dachverbänden der Wirtschaft sowie weiteren, im Einzelfall interessierten Kreisen unterbreitet.","Das Vernehmlassungsverfahren ist eine wichtige Phase bei der Gesetzgebung. In dieser Phase prüft der Bund seine Pläne. Diese Pläne betreffen Politik, Finanzen, Wirtschaft, Umwelt, Soziales oder Kultur. Der Bund will wissen, ob die Pläne richtig sind. Er fragt auch, ob die Pläne durchführbar sind und ob die Menschen sie gut finden. \n\nDazu teilt der Bund die Pläne den Kantonen mit. Kantone sind die grossen Regionen der Schweiz. Der Bund teilt die Pläne auch den Parteien im Parlament mit. Auch die Dachverbände der Gemeinden, Städte und Berggebiete erhalten die Pläne. Die Dachverbände der Wirtschaft bekommen die Pläne ebenfalls. Manchmal interessieren sich noch andere Gruppen für die Pläne. Auch diese anderen Gruppen bekommen dann die Pläne.",2.620196,17.354185,C2,B1
1,gpt-4o,"Das in Artikel 26 aPartG vorgesehene Ehehindernis der eingetragenen Partnerschaft wurde aufgehoben und in Artikel 96 nZGB eingearbeitet. Es ist zu präzisieren, dass das Ehehindernis nur dann besteht, wenn eine frühere Ehe oder eine mit einer Drittperson begründete eingetragene Partnerschaft noch besteht bzw. nicht für ungültig erklärt oder aufgelöst worden ist. Folglich können zwei Personen, die nach Inkrafttreten der ZGB-Revision, d.h. nach dem 30. Juni 2022, im Ausland miteinander eine eingetragene Partnerschaft begründet haben, einander heiraten, sofern die allgemeinen Voraussetzungen von Artikel 43 f. IPRG und des materiellen Schweizer Eherechts erfüllt sind, ohne vorgängig ihre eingetragene Partnerschaft auflösen zu müssen (Bericht RK-N, Ziff. 5.3.3 und 7.1, Ausführungen zu Art. 96 E-ZGB).","Das Gesetz wurde geändert. Früher gab es ein Eheverbot für Personen in einer eingetragenen Partnerschaft. Dieses Verbot steht jetzt in Artikel 96 im Zivilgesetzbuch.\n\nDas Gesetz sagt jetzt: Eine Person darf nur heiraten, wenn keine frühere Ehe oder Partnerschaft besteht. Eine alte Ehe oder Partnerschaft muss entweder aufgehoben oder für ungültig erklärt sein.\n\nSeit dem 30. Juni 2022 ist etwas Neues dazugekommen. Personen, die im Ausland eine eingetragene Partnerschaft geschlossen haben, dürfen trotzdem heiraten. Dafür müssen sie die Regeln in den Artikeln 43 und 96 im Zivilgesetzbuch beachten.\n\nDie Partner müssen die Schweizer Ehe-Regeln beachten. Die Partnerschaft muss nicht vorher aufgelöst werden.\n\nDiese Änderungen stehen in einem Bericht des Nationalrats.",5.120617,16.795429,C2,B1
